In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.utils import *
from keras.initializers import *
import tensorflow as tf
import time, random
import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
batch_size = 64
latent_dim = 256
num_samples = 10000

In [30]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

data_path ='French_english.txt'
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:   # Taking lines from 0 to num_samples which is 10000
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    #print(input_texts, target_texts) #pp
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [120]:
def clean_text(text):
    '''remove punctiations
    remove numbers
    remove contractions
    '''
    
    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [34]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [35]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(                                         #10000, max lenght of input, number of uniq char in inp
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print("encoder_input_data",encoder_input_data.shape)
print("decoder_input_data",decoder_input_data.shape)
print("decoder_target_data",decoder_target_data.shape)
print("**************")

encoder_input_data (10000, 15, 71)
decoder_input_data (10000, 59, 92)
decoder_target_data (10000, 59, 92)
**************


In [61]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.    # earlier we have made a numpy array of zeros so here we are one hot encoding by puttong 1 on places chars are found.
    #encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        #since target_text consit of \t in beginnig we make dec[0,t=0,target_index['\t']] ie dec[0,0,0] =1  \t encoding
        #when t=1 ie v dec_input[o,t=1,'V'] = dec_inp[0,1,42]
        if t > 0:
            #now when t=1 ie V
            #dec[i=0,t=t-1=0,target_ind["V"]] = [0,0,42] =1
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
            #3nd iteration for  'a'
            #dec[o,t=t-1=2-1,target['a']] =[0,1,44] =1

    
'''
looking at dec_inp and dec_out
for V  we see that at dec_input row 1 and col 42 gets encoded, whereas,
dec_target which does not have start and end has row 0 col 42 encoded. 
'''    

In [70]:
input_texts[0], target_texts[0]

('Go.', '\tVa !\n')

In [97]:
input_texts[1], target_texts[1]

('Go.', '\tMarche.\n')

In [80]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [109]:
decoder_input_data[0][1],decoder_target_data[0][0]

#we can see the difference 

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [111]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))             #noen,none,71
encoder = LSTM(latent_dim, return_state=True)#256 dims
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [127]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))   #none.none.92
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)#return both staes and output
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)    #none.none.92
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [129]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=100,
          validation_split=0.2)


Epoch 1/100
125/125 [==============================] - 26s 212ms/step - loss: 0.3673 - accuracy: 0.1912 - val_loss: 0.4733 - val_accuracy: 0.1982
Epoch 2/100
125/125 [==============================] - 27s 212ms/step - loss: 0.3507 - accuracy: 0.1959 - val_loss: 0.4687 - val_accuracy: 0.2010
Epoch 3/100
125/125 [==============================] - 25s 200ms/step - loss: 0.3375 - accuracy: 0.2000 - val_loss: 0.4654 - val_accuracy: 0.2021
Epoch 4/100
125/125 [==============================] - 27s 217ms/step - loss: 0.3244 - accuracy: 0.2040 - val_loss: 0.4604 - val_accuracy: 0.2031
Epoch 5/100
125/125 [==============================] - 24s 188ms/step - loss: 0.3130 - accuracy: 0.2075 - val_loss: 0.4566 - val_accuracy: 0.2064
Epoch 6/100
125/125 [==============================] - 21s 172ms/step - loss: 0.3016 - accuracy: 0.2110 - val_loss: 0.4528 - val_accuracy: 0.2078
Epoch 7/100
125/125 [==============================] - 22s 174ms/step - loss: 0.2910 - accuracy: 0.2137 - val_loss: 0.4516 -

Epoch 57/100
125/125 [==============================] - 41s 327ms/step - loss: 0.0792 - accuracy: 0.2762 - val_loss: 0.6417 - val_accuracy: 0.2108
Epoch 58/100
125/125 [==============================] - 42s 334ms/step - loss: 0.0774 - accuracy: 0.2766 - val_loss: 0.6424 - val_accuracy: 0.2097
Epoch 59/100
125/125 [==============================] - 40s 318ms/step - loss: 0.0763 - accuracy: 0.2769 - val_loss: 0.6521 - val_accuracy: 0.2105
Epoch 60/100
125/125 [==============================] - 41s 324ms/step - loss: 0.0752 - accuracy: 0.2772 - val_loss: 0.6469 - val_accuracy: 0.2107
Epoch 61/100
125/125 [==============================] - 40s 317ms/step - loss: 0.0739 - accuracy: 0.2775 - val_loss: 0.6535 - val_accuracy: 0.2114
Epoch 62/100
125/125 [==============================] - 40s 316ms/step - loss: 0.0724 - accuracy: 0.2778 - val_loss: 0.6613 - val_accuracy: 0.2101
Epoch 63/100
125/125 [==============================] - 39s 316ms/step - loss: 0.0708 - accuracy: 0.2785 - val_loss: 0

In [130]:
model.save('s2s.h5')

In [132]:

from tensorflow import keras
model = keras.models.load_model('s2s.h5')

In [135]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

In [149]:

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [136]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [203]:
def decode_sequence(input_seq):
    # Encode the input as state vectors. ie whole input into context vecots to be fed to decoder
    states_value = encoder_model.predict(input_seq)  # we get context vector

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))   
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.  

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value) 

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :]) #output in [[[]]] hecne [0,0,:]
        sampled_char = reverse_target_char_index[sampled_token_index]#get word fro the index
        decoded_sentence += sampled_char #we get char which gets appended 

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))  #we reintialize seq for next pred
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c] #now instead of encoder states , we senf in h and c generated at respective timestep

    return decoded_sentence



In [205]:

for seq_index in range(3,6):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Filez !

